In [24]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random
import numpy as np

In [4]:
text=open('datasets/SherlockHolmes.txt').read().lower()

text=text[1302:]
print(len(text))

581862
580560


In [6]:
for ch in ['0','1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '"', '$', '%', '&', '~', '`', '(', ')', '*',
          '-', '/', ';', '@', '?', ':', '©', '¢', 'ã', '\xa0', '\n', '\r', '.']:
    if ch in text:
        text=text.replace(ch,' ')
print(set(text))

{'é', 'e', 'a', 'm', 'b', 'à', 'i', 'x', 'r', 'j', ' ', 'n', 't', 'â', 'l', 'w', 'c', ',', 'o', 'f', 'v', 'z', 'p', 'g', 'k', 'y', 'è', 's', 'h', 'd', "'", 'q', 'u'}


In [14]:
window=50
step=3

def window_transform(text,window,step):
    input=[]
    output=[]
    n_batches=int((len(text)-window)/step)
    for i in range(n_batches-1):
        input.append(text[i*step:i*step+window])
        output.append(text[i*step+window])

    return input,output
input,output=window_transform(text,window,step)
input[6],output[6]

('s and predominates the whole of her sex  it was no', 't')

In [22]:
chars=sorted(list(set(text)))
char_to_indices=dict((c,n) for n,c in enumerate(chars))
indices_to_char=dict((n,c) for n,c in enumerate(chars))

In [26]:
def encodeio(text,window,step):
    num_chars=len(chars)
    input,output=window_transform(text,window,step)
    x=np.zeros((len(text),window,num_chars),dtype=bool)
    y=np.zeros((len(text),num_chars),dtype=bool)
    
    for i,sentence in enumerate(input):
        for j,char in enumerate(sentence):
            x[i,j,char_to_indices[char]]=1
        y[i,char_to_indices[output[i]]]=1
    return x,y


x,y=encodeio(text,window,step)
x.shape,y.shape

((580560, 50, 33), (580560, 33))

In [40]:
model = Sequential([
    keras.layers.LSTM(120,input_shape=(window,len(chars))),
    keras.layers.Dropout(0.22),
    keras.layers.Dense(len(chars),activation='linear'),
    keras.layers.Dense(len(chars),activation='softmax')
    
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
x_small=x[:20000,:,:]
y_small=y[:20000,:]
model.fit(x_small,y_small,batch_size=500,epochs=10)


Epoch 1/10
40/40 [==============================] - 1s 12ms/step - loss: 3.0649 - accuracy: 0.1968
Epoch 2/10
40/40 [==============================] - 0s 8ms/step - loss: 2.8431 - accuracy: 0.2174
Epoch 3/10
40/40 [==============================] - 0s 8ms/step - loss: 2.8099 - accuracy: 0.2193
Epoch 4/10
40/40 [==============================] - 0s 8ms/step - loss: 2.7570 - accuracy: 0.2295
Epoch 5/10
40/40 [==============================] - 0s 9ms/step - loss: 2.6761 - accuracy: 0.2411
Epoch 6/10
40/40 [==============================] - 0s 9ms/step - loss: 2.5772 - accuracy: 0.2633
Epoch 7/10
40/40 [==============================] - 0s 8ms/step - loss: 2.4936 - accuracy: 0.2767
Epoch 8/10
40/40 [==============================] - 0s 8ms/step - loss: 2.4347 - accuracy: 0.2841
Epoch 9/10
40/40 [==============================] - 0s 8ms/step - loss: 2.3979 - accuracy: 0.2904
Epoch 10/10
40/40 [==============================] - 0s 7ms/step - loss: 2.3662 - accuracy: 0.2977


In [41]:
def predict_next_chars(model,input_char,num_to_predict):
    pred_text=''
    for i in range(num_to_predict):
        test_x=np.zeros((1,window,len(chars)))
        
        for j, char in enumerate(input_char):
            test_x[0,j,char_to_indices[char]]=1
        test_prediction=model.predict(test_x,verbose=0)[0]

        r=np.argmax(test_prediction)
        d=indices_to_char[r]
        pred_text+=d
        input_char+=d
        input_char=input_char[1:]
    return pred_text


start=89
num_to_predict=10
input_chars=text[start:start+window]
print('complete senquence:',text[start:start+window+num_to_predict])
print('input sequence:',input_chars)
print('output sequence:',predict_next_chars(model,input_chars,num_to_predict))

complete senquence: otion akin to love for irene adler  all emotions, and that o
input sequence: otion akin to love for irene adler  all emotions, 
output sequence: an the  he
